### Checking system requirements

In [ ]:
# connected to NVIDIA drivers with CUDA.
!nvidia-smi

Tue Apr 16 16:45:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Checking OS version
!cat /etc/os-release | head -n2

PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"


### Installing prerequisites

In [ ]:
# clone repository

!git clone https://github.com/weihaox/UMBRAE
!git clone https://github.com/weihaox/BrainHub

Cloning into 'UMBRAE'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 66 (delta 11), reused 63 (delta 8), pack-reused 0
Receiving objects: 100% (66/66), 29.70 MiB | 57.60 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Cloning into 'BrainHub'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 44 (delta 9), reused 44 (delta 9), pack-reused 0
Receiving objects: 100% (44/44), 504.58 KiB | 8.55 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
# install mamba and update base

%%shell
curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
chmod +x Mambaforge-Linux-x86_64.sh
bash Mambaforge-Linux-x86_64.sh -b -f -p /usr/local
mamba config --env --set always_yes true
rm Mambaforge-Linux-x86_64.sh
mamba update -n base -c defaults conda -y

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 80.5M  100 80.5M    0     0  62.4M      0  0:00:01  0:00:01 --:--:-- 77.6M
PREFIX=/usr/local
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2024.2.2-hbcca054_0.conda
Extracting ld_impl_linux-64-2.40-h41732ed_0.conda
Extracting libstdcxx-ng-13.2.0-h7e041cc_5.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.10-4_cp310.conda
Extracting tzdata-2024a-h0c530f3_0.conda
Extracting libgomp-13.2.0-h807b86a_5.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-ng-13.2.0-h807b86a_5.conda
Extracting bzip2-1.0.8-hd590300_5.conda
Extracting c-ares-1.27.0-hd590300_0.conda
Extracting fmt-10.2.1-h00

In [ ]:
%%shell
cd UMBRAE/umbrae
mamba env create -f environment.yaml > /dev/null
mamba init bash
source /root/.bashrc
source activate brainx

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==

Added mamba to /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl.m

In [ ]:
%%shell
source activate brainx
pip install -r requirements.txt

### Downloading Data and Checkpoints

In [ ]:
%%shell
destination="UMBRAE/umbrae/nsd_data"
subdirs=("test") # ("train" "test" "val")
for subdir in "${subdirs[@]}"; do
    full_destination="${destination}/webdataset_avg_split/${subdir}/"
    mkdir -p "$full_destination"
done

declare -a i_values=(1 2 5 7)

# download the test set
for i in "${i_values[@]}"; do
  for j in {0..1}; do
    url="https://huggingface.co/datasets/pscotti/naturalscenesdataset/resolve/main/webdataset_avg_split/test/test_subj0${i}_${j}.tar"
    wget -P "$full_destination" "$url"
  done
done

In [ ]:
%%shell
cd BrainHub
# download reference images
wget -P "caption" "https://huggingface.co/datasets/weihaox/brainx/resolve/main/all_images.pt"
python processing/decode_images.py

In [ ]:
%%capture
# download checkpoints
!python -c  'from huggingface_hub import snapshot_download; snapshot_download(repo_id="weihaox/brainx", repo_type="dataset", local_dir="UMBRAE/umbrae", ignore_patterns=["all_images.pt", ".gitattributes"])'

In [ ]:
# !pip install torch
# !pip install torchvision
# !pip install braceexpand
# !pip install huggingface_hub
# !pip install webdataset
# !pip install transformers
# !pip install einops
# !pip install einops_exts
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
%%shell
prompt_caption='Describe this image <image> as simply as possible.'
prompt_ground='Please interpret this image and give coordinates [x1,y1,x2,y2] for each object you mention.'
cd UMBRAE/umbrae
python inference.py --fmri_encoder 'brainx' --subj 1 --prompt "$prompt_ground" \
    --data_path 'nsd_data' --brainx_path 'train_logs/brainx.pth' \
    --save_path 'evaluation/eval_caption/caption_results/umbrae' --save_image True